In [1]:
import pandas as pd
import numpy as np
import datetime

import re
import requests
from bs4 import BeautifulSoup

D:\anaconda\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


# cleaning and grouping

In [2]:
df = pd.read_csv('history_auction.csv')
df

,auction,date_close_auction,name,year,material,quality,last_bet
0,https://www.wolmar.ru/auction/1700,29.12.2021,1 рубль. Копия,1807.0,"Ag 21,6 гр",UNC,2 084
1,https://www.wolmar.ru/auction/1700,29.12.2021,1 рубль. С подвеса,1812.0,Ag,VF,1 087
2,https://www.wolmar.ru/auction/1700,29.12.2021,"1 рубль Петров - 1,5 рубля.",1841.0,Ag,XF-,5 223
3,https://www.wolmar.ru/auction/1700,29.12.2021,"1 рубль. Отверстие Петров - 1,5 рубля.",1844.0,Ag,F,914
4,https://www.wolmar.ru/auction/1700,29.12.2021,1 рубль,1897.0,Ag,VF/XF,2 738
...,...,...,...,...,...,...,...
1240324,https://www.wolmar.ru/auction/1939,25.01.2024,Рябцевич В.Н. Нумизматика Беларуси.,NaN,Бумага,ХХ,2 491
1240325,https://www.wolmar.ru/auction/1939,25.01.2024,Загорульский Э.М. Вищинский замок XII-XIII вв.,NaN,Бумага,ХХ,342
1240326,https://www.wolmar.ru/auction/1939,25.01.2024,Археология и нумизматика Беларуси.,NaN,Бумага,ХХ,2 491
1240327,https://www.wolmar.ru/auction/1939,25.01.2024,Аверин П.С. Волмар XIII выпуск (июнь 2015) - к...,NaN,Бумага,ХХ,216


In [3]:
df_coins_after_1989 = df[~df['year'].isna()]
df_coins_after_1989['year'] = df_coins_after_1989['year'].astype('int')
df_coins_after_1989 = df_coins_after_1989[df_coins_after_1989['year']>1989]
df_coins_after_1989['date_close_auction'] = pd.to_datetime(df_coins_after_1989['date_close_auction'],
                                                           format='%d.%m.%Y').dt.date
df_coins_after_1989['last_bet'] = df_coins_after_1989['last_bet'].replace(r'\s+','',regex=True).astype('int')

df_coins_after_1989['material_2'] = np.where(df_coins_after_1989['material'].str.startswith('Au', na=False),
                                            'Au',
                                            '-')
df_coins_after_1989['material_2'] = np.where(df_coins_after_1989['material'].str.startswith('Ag', na=False),
                                             'Ag',
                                             df_coins_after_1989['material_2'])
df_coins_after_1989_ag_au = df_coins_after_1989[df_coins_after_1989['material_2']!='-']
df_coins_after_1989_ag_au

<ipython-input-3-55a0dad22dd5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_coins_after_1989['year'] = df_coins_after_1989['year'].astype('int')


,auction,date_close_auction,name,year,material,quality,last_bet,material_2
1287,https://www.wolmar.ru/auction/1700,2021-12-29,1/2 доллара. США,1993,Ag,UNC,761,Ag
1304,https://www.wolmar.ru/auction/1700,2021-12-29,1 доллар. Австралия,2006,"Ag, эмаль",Proof (UNC),3012,Ag
1317,https://www.wolmar.ru/auction/1700,2021-12-29,1 доллар. Австралия,2014,Ag,\tProof (UNC),2577,Ag
1323,https://www.wolmar.ru/auction/1700,2021-12-29,50 центов. Австралия,2003,Ag,Proof (UNC),2522,Ag
1385,https://www.wolmar.ru/auction/1700,2021-12-29,10 динеров. Андорра,2003,Ag,Proof (AU),2822,Ag
...,...,...,...,...,...,...,...,...
1239756,https://www.wolmar.ru/auction/1939,2024-01-25,Жетон Замок Нойшванштайн. Германия,1996,Ag,PF (AU/UNC),1993,Ag
1239757,https://www.wolmar.ru/auction/1939,2024-01-25,Жетон 300 лет Рисвикскому мирному договору. Ни...,1997,Ag,PF (UNC),1506,Ag
1239759,https://www.wolmar.ru/auction/1939,2024-01-25,Жетон Подборка 10 лет Евросоюзу из 2 жетонов,2002,Ag,PF (UNC),2554,Ag
1239760,https://www.wolmar.ru/auction/1939,2024-01-25,Жетон Фаринет. Швейцария,2006,"Au 13,0",PF (AU/UNC),74620,Au


In [4]:
list(df_coins_after_1989_ag_au['quality'].unique())

['UNC',
 'Proof (UNC)',
 '\tProof (UNC)',
 'Proof (AU)',
 'Proof (XF+/AU)',
 'Proof (XF)',
 'AU/UNC',
 'Proof (AU/UNC)',
 'PL (XF)',
 'AU',
 'PF (UNC)',
 'PF (AU/UNC)',
 'PF (AU)',
 'MS 70',
 'PF (XF+/AU)',
 'PF (XF)',
 'PL (UNC)',
 'PL (AU)',
 'PL (XF+/AU)',
 'XF+/AU',
 'PF (XF-)',
 'PF (XF+)',
 'XF',
 'PL (AU/UNC)',
 'PF 70 ULTRA CAMEO ',
 'PF 70 ULTRA CAMEO',
 'PF 70',
 'UNC (Proof)',
 'PROOF',
 'PL AU',
 'Proof-like (AU/UNC)',
 'PL',
 'PF (ХА+/AU)',
 'PF 70 MATTE',
 'PF 69 ULTRA CAMEO',
 'PF 66 ULTRA CAMEO',
 'PF (VF)',
 'PF 69',
 'XF+',
 'BU',
 'PF (UNC-)',
 'PF (VF/XF)',
 'PF 68 ULTRA CAMEO',
 'Proof (XF+)',
 'MS 68',
 'XX',
 'UNC, PF (AU)',
 'PF 65 ULTRA CAMEO',
 'PL (XF+)',
 'MS 69',
 'Proof (XF-)',
 'PF (UNC), UNC',
 'PF 67 ULTRA CAMEO',
 'PF 70 UC',
 'PF 67',
 'VF/XF',
 'PF 69 ULTRA CAMEO ',
 'Proof(UNC)',
 'PF 68',
 'UNC ',
 '\tAU',
 'Proof(AU)',
 'PF 69 ILTRA CAMEO',
 'PR 69 DCAM',
 'PF (ХF+/AU)',
 'PF',
 'PF (AU-UNC), XF-',
 'PF 69 ULTRA CAMEО',
 'PF 68 ULTRA CAMEО',
 '\tP

In [5]:
df_coins_after_1989_ag_au['quality'] = df_coins_after_1989_ag_au['quality'].apply(lambda x: x.replace("\t", ""))
df_coins_after_1989_ag_au['quality'] = df_coins_after_1989_ag_au['quality'].apply(lambda x: x.replace("\n", ""))

<ipython-input-5-62e70479faba>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_coins_after_1989_ag_au['quality'] = df_coins_after_1989_ag_au['quality'].apply(lambda x: x.replace("\t", ""))
<ipython-input-5-62e70479faba>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_coins_after_1989_ag_au['quality'] = df_coins_after_1989_ag_au['quality'].apply(lambda x: x.replace("\n", ""))


In [6]:
list(df_coins_after_1989_ag_au['quality'].unique())

['UNC',
 'Proof (UNC)',
 'Proof (AU)',
 'Proof (XF+/AU)',
 'Proof (XF)',
 'AU/UNC',
 'Proof (AU/UNC)',
 'PL (XF)',
 'AU',
 'PF (UNC)',
 'PF (AU/UNC)',
 'PF (AU)',
 'MS 70',
 'PF (XF+/AU)',
 'PF (XF)',
 'PL (UNC)',
 'PL (AU)',
 'PL (XF+/AU)',
 'XF+/AU',
 'PF (XF-)',
 'PF (XF+)',
 'XF',
 'PL (AU/UNC)',
 'PF 70 ULTRA CAMEO ',
 'PF 70 ULTRA CAMEO',
 'PF 70',
 'UNC (Proof)',
 'PROOF',
 'PL AU',
 'Proof-like (AU/UNC)',
 'PL',
 'PF (ХА+/AU)',
 'PF 70 MATTE',
 'PF 69 ULTRA CAMEO',
 'PF 66 ULTRA CAMEO',
 'PF (VF)',
 'PF 69',
 'XF+',
 'BU',
 'PF (UNC-)',
 'PF (VF/XF)',
 'PF 68 ULTRA CAMEO',
 'Proof (XF+)',
 'MS 68',
 'XX',
 'UNC, PF (AU)',
 'PF 65 ULTRA CAMEO',
 'PL (XF+)',
 'MS 69',
 'Proof (XF-)',
 'PF (UNC), UNC',
 'PF 67 ULTRA CAMEO',
 'PF 70 UC',
 'PF 67',
 'VF/XF',
 'PF 69 ULTRA CAMEO ',
 'Proof(UNC)',
 'PF 68',
 'UNC ',
 'Proof(AU)',
 'PF 69 ILTRA CAMEO',
 'PR 69 DCAM',
 'PF (ХF+/AU)',
 'PF',
 'PF (AU-UNC), XF-',
 'PF 69 ULTRA CAMEО',
 'PF 68 ULTRA CAMEО',
 'MS 69 DCAM',
 'PF (XF',
 'VF+/

In [7]:
df_coins_after_1989_ag_au

,auction,date_close_auction,name,year,material,quality,last_bet,material_2
1287,https://www.wolmar.ru/auction/1700,2021-12-29,1/2 доллара. США,1993,Ag,UNC,761,Ag
1304,https://www.wolmar.ru/auction/1700,2021-12-29,1 доллар. Австралия,2006,"Ag, эмаль",Proof (UNC),3012,Ag
1317,https://www.wolmar.ru/auction/1700,2021-12-29,1 доллар. Австралия,2014,Ag,Proof (UNC),2577,Ag
1323,https://www.wolmar.ru/auction/1700,2021-12-29,50 центов. Австралия,2003,Ag,Proof (UNC),2522,Ag
1385,https://www.wolmar.ru/auction/1700,2021-12-29,10 динеров. Андорра,2003,Ag,Proof (AU),2822,Ag
...,...,...,...,...,...,...,...,...
1239756,https://www.wolmar.ru/auction/1939,2024-01-25,Жетон Замок Нойшванштайн. Германия,1996,Ag,PF (AU/UNC),1993,Ag
1239757,https://www.wolmar.ru/auction/1939,2024-01-25,Жетон 300 лет Рисвикскому мирному договору. Ни...,1997,Ag,PF (UNC),1506,Ag
1239759,https://www.wolmar.ru/auction/1939,2024-01-25,Жетон Подборка 10 лет Евросоюзу из 2 жетонов,2002,Ag,PF (UNC),2554,Ag
1239760,https://www.wolmar.ru/auction/1939,2024-01-25,Жетон Фаринет. Швейцария,2006,"Au 13,0",PF (AU/UNC),74620,Au


In [8]:
gr_coins_material_1_all = df_coins_after_1989_ag_au.groupby(['name', 'year', 'quality', 'material', 'material_2'], 
                                                        as_index=False).last_bet.agg(last_bet_mean_all ='mean',
                                                                                     last_bet_min_all = 'min',
                                                                                     last_bet_max_all = 'max',
                                                                                     count_all = 'count')


gr_coins_material_1_year = df_coins_after_1989_ag_au[df_coins_after_1989_ag_au['date_close_auction']>datetime.datetime.strptime('2023-01-01', '%Y-%m-%d').date()].groupby(['name', 'year', 'quality', 'material', 'material_2'], 
                                                        as_index=False).last_bet.agg(last_bet_mean_year ='mean',
                                                                                     last_bet_min_year = 'min',
                                                                                     last_bet_max_year = 'max',
                                                                                     count_year = 'count')

In [9]:
df_info_coins = pd.merge(gr_coins_material_1_all, 
                         gr_coins_material_1_year, 
                         on=['name','year', 'quality', 'material' , 'material_2'], 
                         how="left")

df_info_coins

,name,year,quality,material,material_2,last_bet_mean_all,last_bet_min_all,last_bet_max_all,count_all,last_bet_mean_year,last_bet_min_year,last_bet_max_year,count_year
0,"0,1 разменный знак. Шпицберген. Артктикуголь",1998,MS 67,Ag,Ag,1749.000000,1749,1749,1,1749.000000,1749.0,1749.0,1.0
1,"0,5 разменный знак. Шпицберген. Артктикуголь",1998,UNC,Ag,Ag,1238.000000,1238,1238,1,1238.000000,1238.0,1238.0,1.0
2,1 000 000 лир. Турция,1997,PF (AU/UNC),"Au 1,25",Au,6798.333333,6032,8213,3,6798.333333,6032.0,8213.0,3.0
3,1 000 000 лир. Турция,1997,UNC,"Au 1,25",Au,6187.666667,6032,6509,6,6187.666667,6032.0,6509.0,6.0
4,1 000 вон. Корея,2006,PF (UNC),Ag,Ag,2147.500000,1594,2721,14,2081.166667,1794.0,2301.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9143,Фаринет. Швейцария,2006,PF (AU/UNC),"Au 13,0",Au,56636.250000,41246,72302,4,NaN,NaN,NaN,NaN
9144,Фунт. Джерси,2007,PF (UNC),"Au 1,26",Au,6507.807692,3567,8942,26,7366.384615,3567.0,8942.0,13.0
9145,Экю. Великобритания,1992,PF (AU),"Au 5,78",Au,29068.153846,14711,40795,26,33794.076923,20609.0,40795.0,13.0
9146,Эскудо. Португалия,2001,UNC,"Au 4,57",Au,20568.000000,13044,26247,9,NaN,NaN,NaN,NaN


# Current auction

In [11]:
auction = []
id_in_auction = []
name_lot = []
year_lot = []
material_lot = []
quality_lot = []
last_bet = []


for number_auction in range(1940,1942):
    req = requests.get("https://www.wolmar.ru/auction/"+str(number_auction))
    soup = BeautifulSoup(req.text, "html.parser")

    if re.search(r'(VIP)', soup.find('h1').text) is not None or re.search(r'(Standart)', soup.find('h1').text) is not None:
        
        for i in soup.find('ul').findAll('li')[::-1]:
            last_page = i.text
            break

        for page in range(1,int(last_page)+1):
            page_req = requests.get("https://www.wolmar.ru/auction/"+str(number_auction)+"?page="+str(page))
            page_soup = BeautifulSoup(page_req.text, "html.parser")
            
            for i in page_soup.findAll('table')[4].findAll('tr')[3:]:
                if i.findAll('td')[1].find('a') is not None:
                    auction.append("https://www.wolmar.ru/auction/"+str(number_auction))
                    id_in_auction.append(i.findAll('td')[0].text)
                    name_lot.append(i.findAll('td')[1].find('a').text)
                    year_lot.append(i.findAll('td')[2].text)
                    material_lot.append(i.findAll('td')[4].text)
                    quality_lot.append(i.findAll('td')[5].find('nobr').text)
                    last_bet.append(i.findAll('td')[8].text)

In [12]:
df_current_auction = pd.DataFrame({'auction': auction,
                   'id' : id_in_auction,
                   'name':name_lot,
                   'year': year_lot,
                   'material':material_lot,
                   'quality': quality_lot,
                   'last_bet':last_bet})
df_current_auction

,auction,id,name,year,material,quality,last_bet
0,https://www.wolmar.ru/auction/1940,2,1 рубль. Ремонт,1828,Ag,VF,2 542
1,https://www.wolmar.ru/auction/1940,3,Полтина. Ремонт,1817,Ag,VF,4
2,https://www.wolmar.ru/auction/1940,4,50 копеек,1897,Ag,VF,2
3,https://www.wolmar.ru/auction/1940,5,50 копеек,1897,Ag,VF+,205
4,https://www.wolmar.ru/auction/1940,6,25 копеек,1850,Ag,VF/XF,2 219
...,...,...,...,...,...,...,...
11320,https://www.wolmar.ru/auction/1941,5137,Кольцо с топазом,,Ag,ХХ,1
11321,https://www.wolmar.ru/auction/1941,5138,Серьги,,Ag,ХХ,1
11322,https://www.wolmar.ru/auction/1941,5139,Кольцо,,Ag,ХХ,1
11323,https://www.wolmar.ru/auction/1941,5140,Серьги,,Ag,ХХ,1


In [13]:
df_current_auction_after_1989 = df_current_auction[df_current_auction['year']!='']
df_current_auction_after_1989['year'] = df_current_auction_after_1989['year'].astype('int')
df_current_auction_after_1989 = df_current_auction_after_1989[df_current_auction_after_1989['year']>1989]
df_current_auction_after_1989['last_bet'] = df_current_auction_after_1989['last_bet'].replace(r'\s+','',regex=True).astype('int')

df_current_auction_after_1989['material_2'] = np.where(df_current_auction_after_1989['material'].str.startswith('Au', na=False),
                                                       'Au',
                                                       '-')
df_current_auction_after_1989['material_2'] = np.where(df_current_auction_after_1989['material'].str.startswith('Ag', na=False),
                                                       'Ag',
                                                       df_current_auction_after_1989['material_2'])
df_current_auction_after_1989 = df_current_auction_after_1989[df_current_auction_after_1989['material_2']!='-']

df_current_auction_after_1989['quality'] = df_current_auction_after_1989['quality'].apply(lambda x: x.replace("\t", ""))
df_current_auction_after_1989['quality'] = df_current_auction_after_1989['quality'].apply(lambda x: x.replace("\n", ""))

<ipython-input-13-9e544cbfaa07>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_current_auction_after_1989['year'] = df_current_auction_after_1989['year'].astype('int')


In [14]:
df_current_auction_after_1989_info = pd.merge(df_current_auction_after_1989, 
                                              df_info_coins, 
                                              on=['name','year', 'quality', 'material', 'material_2'], 
                                              how="left")
df_current_auction_after_1989_info

,auction,id,name,year,material,quality,last_bet,material_2,last_bet_mean_all,last_bet_min_all,last_bet_max_all,count_all,last_bet_mean_year,last_bet_min_year,last_bet_max_year,count_year
0,https://www.wolmar.ru/auction/1940,1101,100 драмов. Армения,2008,"Ag, эмаль",Proof (UNC),1,Ag,3358.142857,3041.0,3512.0,7.0,3358.142857,3041.0,3512.0,7.0
1,https://www.wolmar.ru/auction/1940,1126,20 рублей. Беларусь,2005,"Ag,вставка",UNC,2,Ag,2154.000000,2154.0,2154.0,1.0,NaN,NaN,NaN,NaN
2,https://www.wolmar.ru/auction/1940,1266,10 марок. Германия,1992,Ag,AU/UNC,1,Ag,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://www.wolmar.ru/auction/1940,1267,10 марок. Германия,1998,Ag,UNC,1,Ag,1595.764706,883.0,2406.0,17.0,1635.833333,1104.0,2007.0,12.0
4,https://www.wolmar.ru/auction/1940,1268,10 марок. Германия,2000,Ag,UNC,1,Ag,1417.857143,1128.0,2053.0,7.0,1163.800000,1128.0,1245.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
973,https://www.wolmar.ru/auction/1941,4804,Медаль памятная. Украина,2007,"Ag 31,1",PF (UNC),615,Ag,2388.333333,1646.0,3199.0,42.0,2388.333333,1646.0,3199.0,42.0
974,https://www.wolmar.ru/auction/1941,4873,Жетон 20 марок. Германия,2003,"Au 3,57",PF (AU),1,Au,21496.083333,12129.0,25203.0,12.0,21496.083333,12129.0,25203.0,12.0
975,https://www.wolmar.ru/auction/1941,4874,Жетон 20 марок. Германская Новая Гвинея.,2003,"Au 3,49",PF (AU),3588,Au,19518.500000,9902.0,25397.0,18.0,21531.384615,14268.0,25397.0,13.0
976,https://www.wolmar.ru/auction/1941,4875,Жетон в 5 талеров. Брауншвейг-Ганновер,2004,"Au 3,14",PF (AU),1,Au,17950.222222,11316.0,22550.0,18.0,19025.230769,13741.0,22550.0,13.0


In [15]:
df_current_auction_after_1989_info['price_less_min_y'] = np.where((df_current_auction_after_1989_info['last_bet'] / df_current_auction_after_1989_info['last_bet_min_year']) < 0.8,
                                                                  1,
                                                                  0)

df_current_auction_after_1989_info['price_less_max_y'] = np.where((df_current_auction_after_1989_info['last_bet'] / df_current_auction_after_1989_info['last_bet_max_year']) < 0.8,
                                                                  1,
                                                                  0)

In [16]:
df_current_auction_after_1989_info

,auction,id,name,year,material,quality,last_bet,material_2,last_bet_mean_all,last_bet_min_all,last_bet_max_all,count_all,last_bet_mean_year,last_bet_min_year,last_bet_max_year,count_year,price_less_min_y,price_less_max_y
0,https://www.wolmar.ru/auction/1940,1101,100 драмов. Армения,2008,"Ag, эмаль",Proof (UNC),1,Ag,3358.142857,3041.0,3512.0,7.0,3358.142857,3041.0,3512.0,7.0,1,1
1,https://www.wolmar.ru/auction/1940,1126,20 рублей. Беларусь,2005,"Ag,вставка",UNC,2,Ag,2154.000000,2154.0,2154.0,1.0,NaN,NaN,NaN,NaN,0,0
2,https://www.wolmar.ru/auction/1940,1266,10 марок. Германия,1992,Ag,AU/UNC,1,Ag,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
3,https://www.wolmar.ru/auction/1940,1267,10 марок. Германия,1998,Ag,UNC,1,Ag,1595.764706,883.0,2406.0,17.0,1635.833333,1104.0,2007.0,12.0,1,1
4,https://www.wolmar.ru/auction/1940,1268,10 марок. Германия,2000,Ag,UNC,1,Ag,1417.857143,1128.0,2053.0,7.0,1163.800000,1128.0,1245.0,5.0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
973,https://www.wolmar.ru/auction/1941,4804,Медаль памятная. Украина,2007,"Ag 31,1",PF (UNC),615,Ag,2388.333333,1646.0,3199.0,42.0,2388.333333,1646.0,3199.0,42.0,1,1
974,https://www.wolmar.ru/auction/1941,4873,Жетон 20 марок. Германия,2003,"Au 3,57",PF (AU),1,Au,21496.083333,12129.0,25203.0,12.0,21496.083333,12129.0,25203.0,12.0,1,1
975,https://www.wolmar.ru/auction/1941,4874,Жетон 20 марок. Германская Новая Гвинея.,2003,"Au 3,49",PF (AU),3588,Au,19518.500000,9902.0,25397.0,18.0,21531.384615,14268.0,25397.0,13.0,1,1
976,https://www.wolmar.ru/auction/1941,4875,Жетон в 5 талеров. Брауншвейг-Ганновер,2004,"Au 3,14",PF (AU),1,Au,17950.222222,11316.0,22550.0,18.0,19025.230769,13741.0,22550.0,13.0,1,1


In [17]:
df_current_auction_after_1989_info.to_excel('current_auct.xlsx',
                                             sheet_name='Sheet_name_1',
                                             index=False)

<ipython-input-17-bd9254755790>:1: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.2.9' currently installed).
  df_current_auction_after_1989_info.to_excel('current_auct.xlsx',
